In [1]:
import numpy as np
import tensorflow as tf



In [2]:
def create_model(shape):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64,kernel_size=(4,4),strides=(1,1),activation="relu",input_shape=shape),
    #tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2,2), padding='valid'),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(2,2),strides=(1,1),activation="relu"),
    #tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1,1), padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1),
    
    ])
    
    loss_fn =tf.keras.losses.MeanSquaredError()
    model.compile(optimizer="adam",
              loss=loss_fn,
              metrics=['accuracy'])

    return model

In [3]:
def grad_des(y_hat,y,model):
    model.fit(np.array([y_hat]), np.reshape(y,(1,)) ,verbose=0)
def forward_prop(state,model):
    predictions = model(np.array([state])).numpy()
    return predictions

In [4]:
def argmax(q_values):
    return q_values.index(max(q_values))


In [5]:
class DynaQAgent():
    
    def agent_init(self, agent_info):
        self.input_shape= agent_info.get("input_shape",(10,10,1))
        self.num_actions = agent_info.get("num_actions",4)
        self.gamma = agent_info.get("discount", 0.97)
        self.step_size = agent_info.get("step_size", 0.001)
        self.epsilon = agent_info.get("epsilon", 0.1)
        self.hunger=agent_info.get("hunger", 50)
        self.epsilon_decay=agent_info.get("epsilon_decay",10**-5)
        self.min_epsilon=agent_info.get("min_epsilon",0.001)
        self.modelQueue_capacity=agent_info.get("modelQueue_capacity", 20000)
        self.batch_size=agent_info.get("batch_size", 10)
        self.model_planing_preiod=agent_info.get("model_planing_preiod", 4)
        self.actions = list(range(self.num_actions))
        self.modelQueue = []
        self.models=[]
        self.previous_state=None
        self.last_action=None

        self.iteration_count=1
        for i in range(self.num_actions):
            self.models.append(create_model(self.input_shape))
    def select_action(self, state):
        chosen_action = None
        action_values=[]
        save_epsilon=self.epsilon
        if(self.hunger>50):
            self.epsilon=0.2
        for x in self.models:
            action_values.append(forward_prop(state,x))
        if np.random.random() > self.epsilon: 
            chosen_action = argmax(action_values)
        else: 
            chosen_action = np.random.choice(self.num_actions)
        self.epsilon=save_epsilon
        self.epsilon-=self.epsilon_decay
        self.hunger+=1
        self.epsilon=max(self.min_epsilon,self.epsilon)
        return chosen_action, action_values[chosen_action],max(action_values)
    def model_update(self,state,action,reward,new_state):
        elemant=[(state,action),(reward,new_state)] 
        i=0
        for x in self.modelQueue:
            
            comparison=elemant[0][0]==x[0][0]
            if comparison.all() and elemant[0][1]==x[0][1]:
                self.modelQueue.pop(i)
                self.iteration_count+=1
                self.modelQueue.append(elemant)
                return
            i+=1
        if (len(self.modelQueue))==self.modelQueue_capacity:
            self.modelQueue.pop(0)
        self.modelQueue.append(elemant)
        self.iteration_count+=1
    def planing_step(self):
        batch=np.random.choice(np.array(self.modelQueue).shape[0],self.batch_size)
        for x in batch:
            past_state=self.modelQueue[x][0][0]
            action=self.modelQueue[x][0][1]
            reward=self.modelQueue[x][1][0]
            state=self.modelQueue[x][1][1]
            if(state==-1):
                y=reward
            else:
                y=reward+self.gamma*state
            grad_des(past_state,y,self.models[action])
    def start(self,state):
        self.hunger=0
        current_action , self.lastActionValue,maxActionValue= self.select_action(state)
        self.last_action = current_action
        self.previous_state = state
        return self.last_action
    def step(self,reward,state):
        if reward==1:
            self.hunger=0
        current_action,action_value,maxActionValue=self.select_action(state)
        y=reward+self.gamma*maxActionValue
        grad_des(self.previous_state,y,self.models[self.last_action])
        self.model_update(self.previous_state,self.last_action,reward,maxActionValue)
        if(self.iteration_count%self.model_planing_preiod==0):
            self.planing_step()
        self.last_action=current_action
        self.previous_state=state
        return self.last_action
    def end(self,reward):
        y=reward
        grad_des(self.previous_state,y,self.models[self.last_action])
        self.model_update(self.previous_state,self.last_action,reward,-1)
        if(self.iteration_count%self.model_planing_preiod==0):
            self.planing_step()